# Generalización del entrenamiento

Podemos escribir métodos compartidos que simplifiquen el uso de los distintos tipos de red que necesitamos

Para esto utilizaremos la biblioteca _Lightning_

In [30]:
import os
from torch import optim, nn, utils, backends, cuda, save as torch_save
from torch.amp import autocast, GradScaler
from torchvision.datasets import ImageFolder
from torchvision import models
import lightning as L

def dir_to_dataloader(path, type: str, shuffle: bool = False, transform=None, batch_size = 16):
    train_dataset = ImageFolder(root=os.path.join(path, type), transform=transform)
    # DataLoaders con batch reducido y pin_memory para eficiencia  # Reducido para evitar OOM
    return utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=True)

# Función de creación del modelo
def create_model(model_name, default_weights : bool, num_classes: int):
    # Convertir el nombre del modelo a minúsculas
    model_name_lower = model_name.lower()

    # Intentar importar el modelo dinámicamente usando getattr
    try:
        model_class = getattr(models, model_name_lower)
        weights_class = getattr(models, f"{model_name}_Weights", None)  # Obtener pesos si existen
    except AttributeError:
        raise ValueError(f"Model '{model_name}' not found in torchvision.models")

    # Cargar el modelo con o sin pesos preentrenados
    if default_weights and weights_class:
        model = model_class(weights=weights_class.DEFAULT)  # Si se quieren los pesos preentrenados
    else:
        model = model_class(weights=None)  # Sin pesos preentrenados

    # Ajustar el clasificador al número de clases

    return model

def load_transforms(model_name: str):
    # Intentar importar el modelo dinámicamente usando getattr
    try:
        weights_class = getattr(models, f"{model_name}_Weights", None)  # Obtener pesos si existen
    except AttributeError:
        raise ValueError(f"Model '{model_name}' not found in torchvision.models")
    return weights_class.DEFAULT.transforms()

def train_model(model_name, loader, device, default_weights=False, num_epochs=10):
    num_classes = len(loader.dataset.classes)  

    model = create_model(model_name=model_name, default_weights=default_weights, num_classes=num_classes)
    model = model.to(device)
    # Liberar memoria en la GPU antes de comenzar
    cuda.empty_cache()
    
    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    # Configurar AMP para entrenamiento en precisión mixta
    scaler = GradScaler(enabled=cuda.is_available())

    # Entrenamiento 
    best_acc = 0.0
    save_path = f"../models/{model_name.lower()}/best_model.pth"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Crear directorios si no existen
    print(f"Entrenando {model_name}")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with autocast("cuda" if cuda.is_available() else "cpu", enabled=cuda.is_available()):  # Usar AMP solo si hay GPU
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = 100. * correct / total
        print(f"Epoch {epoch + 1}/{num_epochs}: Loss: {train_loss:.4f} - Accuracy: {train_acc:.2f}%")

        # Guardar el mejor modelo basado en precisión
        if train_acc > best_acc:
            best_acc = train_acc
            torch_save(model.state_dict(), "best_model.pth")
            print("✅ Mejor modelo guardado!")
    print(f"Finalizo entrenamiento de {model_name}")

In [31]:
from torch import cuda, device as torch_device

# Configuración del dispositivo
device = torch_device("cuda" if cuda.is_available() else "cpu")
print("Using: ", device)

# Liberar memoria en la GPU antes de comenzar
cuda.empty_cache()

# Habilitar optimizaciones en GPUs NVIDIA
backends.cudnn.benchmark = True

# Cargar el dataset
data_dir = "../data/osteosarcoma2019"

experiment_models = ["AlexNet", "DenseNet121", "EfficientNet_V2_S", "ResNet18", "VGG11_BN", "ViT_B_16"]
# 
for model_name in experiment_models:
    transform = load_transforms(model_name)
    train_loader = dir_to_dataloader(path=data_dir, type="train", shuffle=True, transform=transform, batch_size=32)
    train_model(model_name, train_loader, device)

Using:  cuda
Entrenando AlexNet
Epoch 1/10: Loss: 2.1881 - Accuracy: 41.14%
✅ Mejor modelo guardado!
Epoch 2/10: Loss: 1.1814 - Accuracy: 40.26%
Epoch 3/10: Loss: 1.1036 - Accuracy: 41.68%
✅ Mejor modelo guardado!
Epoch 4/10: Loss: 1.1018 - Accuracy: 46.50%
✅ Mejor modelo guardado!
Epoch 5/10: Loss: 0.9397 - Accuracy: 51.42%
✅ Mejor modelo guardado!
Epoch 6/10: Loss: 0.9663 - Accuracy: 51.31%
Epoch 7/10: Loss: 0.8638 - Accuracy: 55.03%
✅ Mejor modelo guardado!
Epoch 8/10: Loss: 0.8897 - Accuracy: 56.13%
✅ Mejor modelo guardado!
Epoch 9/10: Loss: 0.8311 - Accuracy: 58.97%
✅ Mejor modelo guardado!
Epoch 10/10: Loss: 0.7629 - Accuracy: 66.41%
✅ Mejor modelo guardado!
Finalizo entrenamiento de AlexNet
Entrenando DenseNet121
Epoch 1/10: Loss: 1.3720 - Accuracy: 75.60%
✅ Mejor modelo guardado!
Epoch 2/10: Loss: 0.4842 - Accuracy: 81.29%
✅ Mejor modelo guardado!
Epoch 3/10: Loss: 0.3585 - Accuracy: 86.65%
✅ Mejor modelo guardado!
Epoch 4/10: Loss: 0.3818 - Accuracy: 86.87%
✅ Mejor modelo guar